In [6]:
import importlib
import functions

importlib.reload(functions)


<module 'functions' from 'c:\\Users\\Thomas\\Desktop\\cpl_analysis_naman\\functions.py'>

Hello Thomas


## 02 Loading the folder that contains our data

In [17]:
import glob
import getpass
user= (getpass.getuser())
print("Hello", user)

base='C:\\Users\\{}\\Dropbox\\CPLab'.format(user)
files = glob.glob(base+'\\all_data_mat_filtered\\*.mat')
savepath = base+'\\results\\'
print(files)

Hello Thomas
['C:\\Users\\Thomas\\Dropbox\\CPLab\\all_data_mat_filtered\\20230529_dk1_nocontext.mat', 'C:\\Users\\Thomas\\Dropbox\\CPLab\\all_data_mat_filtered\\20230529_dk3_nocontext.mat', 'C:\\Users\\Thomas\\Dropbox\\CPLab\\all_data_mat_filtered\\20230529_dk5_nocontext.mat', 'C:\\Users\\Thomas\\Dropbox\\CPLab\\all_data_mat_filtered\\20230529_dk6_nocontext.mat', 'C:\\Users\\Thomas\\Dropbox\\CPLab\\all_data_mat_filtered\\20230531_dk1_nocontext_day2.mat', 'C:\\Users\\Thomas\\Dropbox\\CPLab\\all_data_mat_filtered\\20230531_dk3_nocontext_day2.mat', 'C:\\Users\\Thomas\\Dropbox\\CPLab\\all_data_mat_filtered\\20230531_dk5_nocontext_day2.mat', 'C:\\Users\\Thomas\\Dropbox\\CPLab\\all_data_mat_filtered\\20230531_dk6_nocontext_day2.mat', 'C:\\Users\\Thomas\\Dropbox\\CPLab\\all_data_mat_filtered\\20230609_dk1_BW_nocontext_day1.mat', 'C:\\Users\\Thomas\\Dropbox\\CPLab\\all_data_mat_filtered\\20230609_dk3_BW_nocontext_day1.mat', 'C:\\Users\\Thomas\\Dropbox\\CPLab\\all_data_mat_filtered\\20230610_dk

In [9]:
keyboard_dict={'98':'b','119':'w','120':'nc','49':'1','48':'0'}

## 03  Looping through all the data files

In [16]:
import os
import h5py
import numpy as np
import pandas as pd
import pickle

files=[f'C:\\Users\\{user}\\Dropbox\\CPLab\\all_data_mat_filtered\\20230615_dk6_BW_context_day1.mat'] #This is just for testing purposes
events_codes_all = {}
for file in files:
    # Initialize an empty DataFrame to store the results for all epochs
    compiled_data_all_epochs = pd.DataFrame()
    ## Get the date, mouse_id and task from the file name
    base_name = os.path.basename(file)
    base_name, _ = os.path.splitext(base_name)

    date, mouse_id, task=functions.exp_params(base_name)
    print(date, mouse_id, task)
    ## Open the file
    f=h5py.File(file, 'r')
    channels = list(f.keys())
    print(channels)

    if 'Keyboard' in channels:
        events = f['Keyboard']
    elif 'keyboard' in channels:
        events = f['keyboard']
    elif 'memory' in channels:
        events = f['memory']
    elif 'Memory' in channels:
        events = f['Memory']

    events_codes=np.array(events['codes'][0])
    events_times=np.array(events['times'][0])
    events_codes_all[base_name] = events_codes
    #Generating epochs from events (epochs are basically start of a trial and end of a trial)
    epochs=functions.generate_epochs_with_first_event(events_codes, events_times)

    # Experiment Start time
    first_event=events_times[0]

    for channeli in channels:
        if "AON" in channeli or  "vHp" in channeli :
            
            channel_id=channeli
            # Extracting raw data and time
            data_all=f[channeli]
            raw_data=np.array(data_all['values']).flatten()
            raw_time = np.array(data_all['times']).flatten()
            sampling_rate = int(1/data_all['interval'][0][0])
            print(raw_data.shape, raw_time.shape, sampling_rate)
            # Normalizing the data by subtracting the mean and std of data 30sec before the first event
            normalized_data,time=functions.data_normalization(raw_data,raw_time,first_event, sampling_rate)

            # Applying a notch filter
            notch_filtered_data=functions.iir_notch(normalized_data, sampling_rate,60)
            total=notch_filtered_data
            # Extracting the bands
            beta=functions.beta_band(notch_filtered_data, sampling_rate)
            gamma=functions.gamma_band(notch_filtered_data, sampling_rate)
            theta=functions.theta_band(notch_filtered_data, sampling_rate)

            all_bands=[total,beta, gamma, theta]

            for i,epochi in enumerate(epochs):
                compiled_data = pd.DataFrame()

                trial_timestamp = epochi[0][0]
                trial_type = epochi[0][1]
                dig_type = epochi[1, 1]
                print(dig_type)
                dig_timestamp = epochi[1, 0]
                print(trial_timestamp,trial_type,dig_timestamp,dig_type)
                for bandi in all_bands:
                    data_trial_before, data_trial_after=functions.extract_trial_data(bandi,time,trial_timestamp,sampling_rate)
                    data_dig_before, data_dig_after=functions.extract_dig_data(bandi,time,dig_timestamp,sampling_rate)

                                        # Create a DataFrame for the current bandi
                    bandi_data = pd.DataFrame({
                        'data_trial_before': [data_trial_before],
                        'data_trial_after': [data_trial_after],
                        'data_dig_before': [data_dig_before],
                        'data_dig_after': [data_dig_after]
                    })
                    
                    # Concatenate the current bandi DataFrame with the compiled_data DataFrame along axis=1
                    compiled_data = pd.concat([compiled_data, bandi_data], axis=1)
                compiled_data.columns = ['total_pre_door', 'total_post_door', 'total_pre_odor', 'total_post_odor',
                            'beta_pre_door', 'beta_post_door', 'beta_pre_odor', 'beta_post_odor',
                            'gamma_pre_door', 'gamma_post_door', 'gamma_pre_odor', 'gamma_post_odor',
                            'theta_pre_door', 'theta_post_door', 'theta_pre_odor', 'theta_post_odor']
                compiled_data.insert(0, 'rat', mouse_id)
                compiled_data.insert(1, 'date', date)
                compiled_data.insert(2, 'experiment', task)
                compiled_data.insert(3, 'channel', channel_id)
                compiled_data.insert(4, 'trial', i)
                compiled_data.insert(5, 'side', keyboard_dict[str(int(trial_type))])
                compiled_data.insert(6, 'correct?', keyboard_dict[str(int(dig_type))])

                compiled_data_all_epochs = pd.concat([compiled_data_all_epochs, compiled_data], axis=0, ignore_index=True)
    
    #compiled_data_all_epochs.to_csv(f'{base_name}_compiled_data.csv', index=False)
    with open(os.path.join(savepath, f'{base_name}_compiled_data.pkl'), 'wb') as file:
        pickle.dump(compiled_data_all_epochs, file)
                # Extracting data around events
    #         
    # data_around_events=functions.data_events_extract(notch_filtered_data,time,epochs,sampling_rate)

    #         beta_around_events=functions.data_events_extract(beta,time,epochs,sampling_rate)
    #         gamma_around_events=functions.data_events_extract(gamma,time,epochs,sampling_rate)
    #         theta_around_events=functions.data_events_extract(theta,time,epochs,sampling_rate)

    #         # Creating a dataframe
    #         new_row=pd.DataFrame({'experiment_id':base_name,'channel_id':channel_id,
    #                             'epochs':[epochs],'first_event':[first_event],
    #                             'data_around_events':[data_around_events],
    #                             'beta_around_events':[beta_around_events],
    #                             'gamma_around_events':[gamma_around_events],
    #                             'theta_around_events':[theta_around_events]})
    #         try:
    #             compiled_data=pd.concat([compiled_data,new_row],ignore_index=True)
    #         except Exception as e:
    #             print(e)
    #     import pickle

    # #Save the DataFrame to a pickle file
    # with open(f'{base_name}_compiled_data.pkl', 'wb') as file:
    #     pickle.dump(compiled_data, file)

20230615 dk6 BWcontext
['Keyboard', 'LFP1_vHp', 'LFP2_vHp', 'LFP3_AON', 'LFP4_AON', 'Ref', 'Respirat', 'file']
(2695355,) (2695355,) 2000
normalizing data
notch filter applied
filtering beta band
filtering gamma band
filtering theta band
48.0
89.50689499999999 98.0 98.550895 48.0
Extracted trial data from index 179014 to 183014
Extracted dig data before from index 193102 to 197102
Extracted dig data after from index 197102 to 201102
Extracted trial data from index 179014 to 183014
Extracted dig data before from index 193102 to 197102
Extracted dig data after from index 197102 to 201102
Extracted trial data from index 179014 to 183014
Extracted dig data before from index 193102 to 197102
Extracted dig data after from index 197102 to 201102
Extracted trial data from index 179014 to 183014
Extracted dig data before from index 193102 to 197102
Extracted dig data after from index 197102 to 201102
49.0
328.924895 98.0 331.3649 49.0
Extracted trial data from index 657850 to 661850
Extracted d

In [15]:
print(savepath)

C:\Users\Thomas\Dropbox\CPLab\results\


In [49]:
import os
import h5py
import numpy as np
import pandas as pd

files=['C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat\\20230529_dk1_nocontext.mat']
events_codes_all = {}
for file in files:
    compiled_data=pd.DataFrame(columns=['experiment_id','channel_id',
                                    'epochs','first_event',
                                    'data_around_events',
                                    'beta_around_events','gamma_around_events','theta_around_events'])
    ## Get the date, mouse_id and task from the file name
    base_name = os.path.basename(file)
    base_name, _ = os.path.splitext(base_name)

    date, mouse_id, task=functions.exp_params(base_name)
    print(date, mouse_id, task)
    ## Open the file
    f=h5py.File(file, 'r')
    channels = list(f.keys())
    print(channels)

    if 'Keyboard' in channels:
        events = f['Keyboard']
    elif 'keyboard' in channels:
        events = f['keyboard']
    elif 'memory' in channels:
        events = f['memory']
    elif 'Memory' in channels:
        events = f['Memory']

    events_codes=np.array(events['codes'][0])
    events_times=np.array(events['times'][0])
    events_codes_all[base_name] = events_codes
    #Generating epochs from events (epochs are basically start of a trial and end of a trial)
    epochs=functions.generate_epochs_with_first_event(events_codes, events_times)

    # Experiment Start time
    first_event=events_times[0]

    for channeli in channels:
        if "AON" in channeli:
            area='AON'
            print(channeli)
        elif "vHp" in channeli:
            area='vHp'
            print(channeli)
            pass
        else:
            continue
        channel_id=channeli
        # Extracting raw data and time
        data_all=f[channeli]
        raw_data=np.array(data_all['values']).flatten()
        raw_time = np.array(data_all['times']).flatten()
        sampling_rate = int(1/data_all['interval'][0][0])
        print(raw_data.shape, raw_time.shape, sampling_rate)
        # Normalizing the data by subtracting the mean and std of data 30sec before the first event
        normalized_data,time=functions.data_normalization(raw_data,raw_time,first_event, sampling_rate)

        # Applying a notch filter
        notch_filtered_data=functions.iir_notch(normalized_data, sampling_rate,60)

        # Extracting the bands
        beta=functions.beta_band(notch_filtered_data, sampling_rate)
        gamma=functions.gamma_band(notch_filtered_data, sampling_rate)
        theta=functions.theta_band(notch_filtered_data, sampling_rate)

        
        all_bands=[beta, gamma, theta]

        # Extracting data around events
        data_around_events=functions.data_events_extract(notch_filtered_data,time,epochs,sampling_rate)

        beta_around_events=functions.data_events_extract(beta,time,epochs,sampling_rate)
        gamma_around_events=functions.data_events_extract(gamma,time,epochs,sampling_rate)
        theta_around_events=functions.data_events_extract(theta,time,epochs,sampling_rate)


        # Creating a dataframe
        new_row=pd.DataFrame({'experiment_id':base_name,'channel_id':channel_id,
                             'epochs':[epochs],'first_event':[first_event],
                             'data_around_events':[data_around_events],
                             'beta_around_events':[beta_around_events],
                             'gamma_around_events':[gamma_around_events],
                             'theta_around_events':[theta_around_events]})
        try:
            compiled_data=pd.concat([compiled_data,new_row],ignore_index=True)
        except Exception as e:
            print(e)
    import pickle

    #Save the DataFrame to a pickle file
    with open(f'{base_name}_compiled_data.pkl', 'wb') as file:
        pickle.dump(compiled_data, file)

20230529 dk1 nocontext
['Keyboard', 'LFP1_AON', 'LFP1_vHp', 'LFP2_AON', 'LFP2_vHp', 'LFP3_AON', 'LFP4_AON', 'Ref', 'Respirat', 'file']
LFP1_AON
(1516022,) (1516022,) 2000
normalizing data
notch filter applied
filtering beta band
filtering gamma band
filtering theta band
Processing epoch: [[181.203855 120.      ]
 [182.713855  49.      ]]
Processing epoch with trial timestamp 181.20385499999998 and trial type 120.0
Extracted trial data from index 362408 to 366408
Extracted dig data before from index 361428 to 365428
Extracted dig data after from index 365428 to 369428
Processing epoch: [[237.771855 120.      ]
 [239.461855  48.      ]]
Processing epoch with trial timestamp 237.771855 and trial type 120.0
Extracted trial data from index 475544 to 479544
Extracted dig data before from index 474924 to 478924
Extracted dig data after from index 478924 to 482924
Processing epoch: [[279.551855 120.      ]
 [280.29185   49.      ]]
Processing epoch with trial timestamp 279.551855 and trial typ

C:\Users\sinha\AppData\Local\Temp\ipykernel_32920\1802872135.py:87: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  compiled_data=pd.concat([compiled_data,new_row],ignore_index=True)


filtering theta band
Processing epoch: [[181.203855 120.      ]
 [182.713855  49.      ]]
Processing epoch with trial timestamp 181.20385499999998 and trial type 120.0
Extracted trial data from index 362408 to 366408
Extracted dig data before from index 361428 to 365428
Extracted dig data after from index 365428 to 369428
Processing epoch: [[237.771855 120.      ]
 [239.461855  48.      ]]
Processing epoch with trial timestamp 237.771855 and trial type 120.0
Extracted trial data from index 475544 to 479544
Extracted dig data before from index 474924 to 478924
Extracted dig data after from index 478924 to 482924
Processing epoch: [[279.551855 120.      ]
 [280.29185   49.      ]]
Processing epoch with trial timestamp 279.551855 and trial type 120.0
Extracted trial data from index 559104 to 563104
Extracted dig data before from index 556584 to 560584
Extracted dig data after from index 560584 to 564584
Processing epoch: [[348.953855 120.      ]
 [349.84385   48.      ]]
Processing epoch 

In [50]:
data_around_events=functions.data_events_extract(beta,time,epochs,sampling_rate)
print(data_around_events)

Processing epoch: [[181.203855 120.      ]
 [182.713855  49.      ]]
Processing epoch with trial timestamp 181.20385499999998 and trial type 120.0
Extracted trial data from index 362408 to 366408
Extracted dig data before from index 361428 to 365428
Extracted dig data after from index 365428 to 369428
Processing epoch: [[237.771855 120.      ]
 [239.461855  48.      ]]
Processing epoch with trial timestamp 237.771855 and trial type 120.0
Extracted trial data from index 475544 to 479544
Extracted dig data before from index 474924 to 478924
Extracted dig data after from index 478924 to 482924
Processing epoch: [[279.551855 120.      ]
 [280.29185   49.      ]]
Processing epoch with trial timestamp 279.551855 and trial type 120.0
Extracted trial data from index 559104 to 563104
Extracted dig data before from index 556584 to 560584
Extracted dig data after from index 560584 to 564584
Processing epoch: [[348.953855 120.      ]
 [349.84385   48.      ]]
Processing epoch with trial timestamp 

############################ End of Code #####################################

In [51]:
!pip install h5py


In [52]:
testing=functions.data_events_extract(, )
print(testing)

SyntaxError: invalid syntax (3488078390.py, line 1)

In [ ]:
testing=compiled_data.iloc[0]['data_around_events']
print(len(testing))

In [ ]:
compiled_data['experiment_id']=compiled_data['date']+'_'+compiled_data['mouse_id']+'_'+compiled_data['task']
dig_data=pd.DataFrame(columns=['bin_id','trial_id','correct?','total_power','beta_power','gamma_power','theta_power'])
door_data=pd.DataFrame(columns=['bin_id','trial_id','correct?','total_power','beta_power','gamma_power','theta_power'])

all_experiments=list(compiled_data['experiment_id'].unique())
print(all_experiments)
for experiment in all_experiments:
    print(experiment)
    experiment_data=compiled_data[compiled_data['experiment_id']==experiment]
    trial_id=experiment
    channel_ids=experiment_data['channel_id'].unique()
    for channel_idi in channel_ids:
        print(channel_idi)
        channel_data=experiment_data[experiment_data['channel_id']==channel_idi]
        
        data_around_events=channel_data['data_around_events'].iloc[0]    
        
        white_trials=data_around_events[0]
        black_trials=data_around_events[1]
        no_context_trials=data_around_events[3]
        

        white_trials=data_around_events[0]
        black_trials=data_around_events[1]

        correct_in_white_before=data_around_events[2]
        incorrect_in_white_before=data_around_events[3]
        correct_in_white_after=data_around_events[4]
        incorrect_in_white_after=data_around_events[5]

        correct_in_black_before=data_around_events[6]
        incorrect_in_black_before=data_around_events[7]
        correct_in_black_after=data_around_events[8]
        incorrect_in_black_after=data_around_events[9]

        
        #correct_in_white_before=channel_data['data_around_events'][2]    
        #correct_in_black_before=channel_data['data_around_events'][6]

    #correct_trials_all=correct_in_white_before.append(correct_in_black_before)
    #print(correct_trials_all.shape)

In [ ]:
import matplotlib.pyplot as plt
type(data_around_events[2])
print(data_around_events[2].shape)
row_averages = np.mean(data_around_events[6], axis=0)
plt.plot(row_averages)

In [ ]:
plot_testing=compiled_data.iloc[0]['data_around_events'][2]
plot_testing=compiled_data.iloc[1]['data_around_events'][2]

In [ ]:
import numpy as np

def generate_epochs(events, time):
    events_concat = np.vstack((events, time)).T
    print("Events Concat:", events_concat)  # Debugging print

    epochs = []

    # Initialize variables to track the start of an epoch
    start_index = None

    # Iterate through events_concat
    for i, event in enumerate(events_concat):
        print("Event:", event)  # Debugging print
        if event[0] == 119 or event[0] == 98:
            if start_index is None:
                # Start a new epoch
                start_index = i
            else:
                # End the current epoch
                end_index = i
                epoch = events_concat[start_index:end_index]
                epochs.append(epoch)
                start_index = None
    return epochs

# Example usage
events = np.array([1, 2, 119, 3, 4, 98, 5, 6, 119, 7, 8, 98])
time = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120])
testing = generate_epochs(events, time)
print("Generated Epochs:", testing)

In [ ]:
try:
    for epochi in epochs:
        trial_timestamp = epochi[0][0]
        print(trial_timestamp)  
except Exception as e:
    print(e)